In [1]:
import pandas as pd
import dask.dataframe as dd
import dask
import coiled
import dask.distributed

In [2]:
cluster = coiled.Cluster(configuration="coiled/default", n_workers=10)

Output()

Found software environment build


In [3]:
client = dask.distributed.Client(cluster)

In [7]:
def create_by_partition_freq(partition_freq, years):
    for year in years:
        ddf = dask.datasets.timeseries(
            start=f'{year}-01-01',
            end=f'{year}-12-31',
            freq='1s',
            partition_freq=partition_freq,
            seed=42,
        )
        ddf.to_parquet(f"s3://coiled-datasets/timeseries/{partition_freq}/parquet/{year}")
        ddf.to_csv(f"s3://coiled-datasets/timeseries/{partition_freq}/csv/{year}/*.csv")

In [8]:
create_by_partition_freq("7d", ["2000", "2001"])

In [9]:
create_by_partition_freq("1h", ["2000", "2001"])

In [10]:
create_by_partition_freq("7d", ["2002", "2003", "2004", "2005"])

In [14]:
def create_1y_files(years):
    for year in years:
        ddf = dask.datasets.timeseries(
            start=f'{year}-01-01',
            end=f'{year}-12-31',
            freq='1s',
            partition_freq="7d",
            seed=42,
        )
        ddf.repartition(1).to_parquet(f"s3://coiled-datasets/timeseries/1y/parquet/{year}")
        ddf.repartition(1).to_csv(f"s3://coiled-datasets/timeseries/1y/csv/{year}/*.csv")

In [15]:
create_1y_files([2006])

In [18]:
create_1y_files([2007])

CancelledError: list-2d0255d9-1ccc-42cd-8ff8-da565108b2f7